# General Neural Network

In [1]:
source = "GNN";

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
device = gpu;

In [5]:
read_params("GNN.1")["residual_alphas"]

1-element Vector{String}:
 "UserItemBiases"

## Data preparation

In [6]:
function get_data(alpha)
    training = get_residuals("recommendee", read_params(alpha)["residual_alphas"])
    implicit = get_implicit_list()
    n_items = num_items() + 1

    X1 = zeros(Float32, n_items)
    X1[training.item] = training.rating
    X2 = zeros(Float32, n_items)
    X2[implicit.item] = implicit.rating

    features = [
        length(training.item) / n_items,
        length(implicit.item) / n_items,
        sum(get_split("recommendee").rating) / length(training.item) / 10,
    ]
    features = convert.(Float32, features)
    X3 = vcat(features, features .^ 2, sqrt.(features))

    X = vcat(X1, X2, X3)
end;

In [7]:
function get_alphas()
    [
        x for x in read_params("CombineSignals")["alphas"] if
        startswith(x, "GNN") && !endswith(x, ".Implicit")
    ]
end;

In [8]:
function compute_alpha(alpha)
    params = read_params(alpha)
    BSON.@load params["model"] m
    if occursin("Rating", alpha)
        ratings = m(get_data(alpha))
        write_recommendee_alpha(x -> ratings[x], outdir = "$alpha")
    elseif occursin("Implicit", alpha)
        implicit = m(get_data(alpha))
        softmax = exp.(implicit)
        softmax = softmax ./ sum(softmax)
        write_recommendee_alpha(x -> softmax[x], outdir = "$alpha")
    else
        ratings, implicit = m(get_data(alpha))
        softmax = exp.(implicit)
        softmax = softmax ./ sum(softmax)
        write_recommendee_alpha(x -> ratings[x], outdir = "$alpha")
        write_recommendee_alpha(x -> softmax[x], outdir = "$alpha.Implicit")
    end
end;

In [9]:
@showprogress for alpha in get_alphas()
    if should_compute(alpha)
        compute_alpha(alpha)
    end
end;

Progress: 100%|█████████████████████████████████████████| Time: 0:01:01
